In [59]:
from __future__ import annotations
from enum import Enum

class State(Enum):
    off = False
    on = True
    
    def switch(self) -> State:
        if self == State.off:
            return State.on
        else:
            return State.off

class Pulse(Enum):
    low = False
    high = True    
    
    def __str__(self):
        return 'low' if self == Pulse.low else 'high'
    
    def __repr__(self) -> str:
        return self.__str__()
    
class Module:
    
    def recive(self, pulse: Pulse, origin: str) -> Pulse:
        raise NotImplementedError

class FlipFlop(Module):
    
    def __init__(self, name: str):
        self.name = name
        self.state = State.off
        
    def recive(self, pulse: Pulse, origin: str) -> Pulse | None:
        if pulse == Pulse.high:
            return None
        
        self.state = State.switch(self.state)
        if self.state == State.on:
            return Pulse.high
        else:
            return Pulse.low
        
    def __str__(self):
        return f"%{self.name}:{'on' if self.state == State.on else 'off'} "
    
    def __repr__(self) -> str:
        return self.__str__()
        
class Conjunction(Module):
    
    def __init__(self, name: str, inputs: list[str]):
        self.name = name
        self.inputs = {input: Pulse.low for input in inputs}
        
    def recive(self, pulse: Pulse, origin: str) -> Pulse:
        self.inputs[origin] = pulse
        if all(pulse == Pulse.high for pulse in self.inputs.values()):
            return Pulse.low
        return Pulse.high
    
    def __str__(self) -> str:
        return f"&{self.name}:{self.inputs}"
    
    def __repr__(self) -> str:
        return self.__str__()

class Bordcaster(Module):
    
    def recive(self, pulse: Pulse, origin: str) -> Pulse:
        return pulse
    
    def __str__(self):
        return "broadcaster"
    def __repr__(self) -> str:
        return self.__str__()
        

class Model:
    
    def __init__(self, lines: list[str]):
        self.modules: dict[str, tuple[Module, list[str]]] = {}
        
        m = {}
        cons = []
        
        for line in lines:
            parts = line.strip().split(" -> ")
            if line.startswith("broadcaster"): 
                recivers = parts[1].split(", ")
                m['broadcaster'] = recivers
                self.modules['broadcaster'] = (Bordcaster(), recivers)
                
            elif line.startswith("%"):
                name = line[1:line.index(' ')]
                recivers = parts[1].split(", ")
                m[name] = recivers
                self.modules[name] = (FlipFlop(name), recivers)
                
            elif line.startswith("&"):
                name = line[1:line.index(' ')]
                recivers = parts[1].split(", ")
                m[name] = recivers
                cons.append(name)
        
        for name in cons:
            inputs = []
            for key, values in m.items():
                if name in values:
                    inputs.append(key)
            
            self.modules[name] = (Conjunction(name, inputs), m[name])
            
        print(self.modules)
        
    def push_button(self) -> int:
        pulse_cnt = {Pulse.high: 0, Pulse.low: 0}
        
        current_pulses = [('button', 'broadcaster', Pulse.low)]
        
        while len(current_pulses):
            next_pulses = []
            
            for origin, reciver, pulse in current_pulses:
                pulse_cnt[pulse] += 1
                if reciver not in self.modules:
                    continue
                
                next_pulse = self.modules[reciver][0].recive(pulse, origin)
                
                if next_pulse is not None:
                    for next_reciver in self.modules[reciver][1]:
                        next_pulses.append((reciver, next_reciver, next_pulse))                    
                    
            current_pulses = next_pulses
        
        return pulse_cnt

    
with open("input.txt") as f:
    lines = f.readlines()

m = Model(lines)
gcnt = {Pulse.low: 0, Pulse.high: 0}
for _ in range(1000):
    r = m.push_button()
    gcnt[Pulse.high] += r[Pulse.high]
    gcnt[Pulse.low] += r[Pulse.low]


print(gcnt[Pulse.high] * gcnt[Pulse.low])

{'rx': (<__main__.Rx object at 0x7f928c4ea4b0>, ()), 'bd': (%bd:off , ['lg', 'cm']), 'broadcaster': (broadcaster, ['ct', 'hr', 'ft', 'qm']), 'bh': (%bh:off , ['lj']), 'rn': (%rn:off , ['ml']), 'ks': (%ks:off , ['xk', 'kd']), 'gk': (%gk:off , ['qv', 'kd']), 'lg': (%lg:off , ['cm']), 'qd': (%qd:off , ['jp', 'cm']), 'jk': (%jk:off , ['fs']), 'vq': (%vq:off , ['qj', 'kd']), 'lj': (%lj:off , ['cx', 'zz']), 'th': (%th:off , ['bh']), 'rp': (%rp:off , ['bk', 'cm']), 'xk': (%xk:off , ['nv', 'kd']), 'qv': (%qv:off , ['ks']), 'mj': (%mj:off , ['xg', 'mh']), 'lh': (%lh:off , ['zz', 'pz']), 'kb': (%kb:off , ['hv', 'kd']), 'pg': (%pg:off , ['lz']), 'qm': (%qm:off , ['kb', 'kd']), 'pc': (%pc:off , ['cm', 'xl']), 'hv': (%hv:off , ['kd', 'rn']), 'fr': (%fr:off , ['fl', 'mh']), 'mp': (%mp:off , ['zz']), 'xl': (%xl:off , ['cm', 'gb']), 'tp': (%tp:off , ['mh']), 'gb': (%gb:off , ['rp']), 'pz': (%pz:off , ['mf', 'zz']), 'qn': (%qn:off , ['cm', 'sk']), 'fl': (%fl:off , ['tp', 'mh']), 'zq': (%zq:off , ['th',

In [68]:
from __future__ import annotations
from enum import Enum

class State(Enum):
    off = False
    on = True
    
    def switch(self) -> State:
        if self == State.off:
            return State.on
        else:
            return State.off

class Pulse(Enum):
    low = False
    high = True    
    
    def __str__(self):
        return 'low' if self == Pulse.low else 'high'
    
    def __repr__(self) -> str:
        return self.__str__()
    
class Module:
    
    def recive(self, pulse: Pulse, origin: str) -> Pulse:
        raise NotImplementedError

class FlipFlop(Module):
    
    def __init__(self, name: str):
        self.name = name
        self.state = State.off
        
    def recive(self, pulse: Pulse, origin: str) -> Pulse | None:
        if pulse == Pulse.high:
            return None
        
        self.state = State.switch(self.state)
        if self.state == State.on:
            return Pulse.high
        else:
            return Pulse.low
        
    def __str__(self):
        return f"%{self.name}:{'on' if self.state == State.on else 'off'} "
    
    def __repr__(self) -> str:
        return self.__str__()
        
class Conjunction(Module):
    
    def __init__(self, name: str, inputs: list[str]):
        self.name = name
        self.inputs = {input: Pulse.low for input in inputs}
        
    def recive(self, pulse: Pulse, origin: str) -> Pulse:
        self.inputs[origin] = pulse
        if all(pulse == Pulse.high for pulse in self.inputs.values()):
            return Pulse.low
        return Pulse.high
    
    def __str__(self) -> str:
        return f"&{self.name}:{self.inputs}"
    
    def __repr__(self) -> str:
        return self.__str__()

class Bordcaster(Module):
    
    def recive(self, pulse: Pulse, origin: str) -> Pulse:
        return pulse
    
    def __str__(self):
        return "broadcaster"
    def __repr__(self) -> str:
        return self.__str__()
    
class Rx(Module):
    
    def recive(self, pulse: Pulse, origin: str) -> Pulse:
        if pulse == Pulse.low:
            return 'DONE'
        
        return None
        

class Model:
    
    def __init__(self, lines: list[str]):
        self.modules: dict[str, tuple[Module, list[str]]] = {'rx': (Rx(), ())}
        self.foundrx = False
        
        m = {}
        cons = []
        
        for line in lines:
            parts = line.strip().split(" -> ")
            if line.startswith("broadcaster"): 
                recivers = parts[1].split(", ")
                m['broadcaster'] = recivers
                self.modules['broadcaster'] = (Bordcaster(), recivers)
                
            elif line.startswith("%"):
                name = line[1:line.index(' ')]
                recivers = parts[1].split(", ")
                m[name] = recivers
                self.modules[name] = (FlipFlop(name), recivers)
                
            elif line.startswith("&"):
                name = line[1:line.index(' ')]
                recivers = parts[1].split(", ")
                m[name] = recivers
                cons.append(name)
        
        for name in cons:
            inputs = []
            for key, values in m.items():
                if name in values:
                    inputs.append(key)
            
            self.modules[name] = (Conjunction(name, inputs), m[name])
            
        print(self.modules)
        
    def push_button(self) -> int:
        pulse_cnt = {Pulse.high: 0, Pulse.low: 0}
        
        current_pulses = [('button', 'broadcaster', Pulse.low)]
        
        while len(current_pulses):
            next_pulses = []
            
            for origin, reciver, pulse in current_pulses:
                pulse_cnt[pulse] += 1
                
                next_pulse = self.modules[reciver][0].recive(pulse, origin)
                
                if next_pulse == 'DONE':
                    self.foundrx = True
                    return
                elif next_pulse is not None:
                    for next_reciver in self.modules[reciver][1]:
                        next_pulses.append((reciver, next_reciver, next_pulse))                    
                    
            current_pulses = next_pulses
        
        return pulse_cnt

    
with open("input.txt") as f:
    lines = f.readlines()

m = Model(lines)
cnt = 0
while not m.foundrx:
    m.push_button()
    cnt += 1

print(cnt)

{'rx': (<__main__.Rx object at 0x7f928c4ec2c0>, ()), 'bd': (%bd:off , ['lg', 'cm']), 'broadcaster': (broadcaster, ['ct', 'hr', 'ft', 'qm']), 'bh': (%bh:off , ['lj']), 'rn': (%rn:off , ['ml']), 'ks': (%ks:off , ['xk', 'kd']), 'gk': (%gk:off , ['qv', 'kd']), 'lg': (%lg:off , ['cm']), 'qd': (%qd:off , ['jp', 'cm']), 'jk': (%jk:off , ['fs']), 'vq': (%vq:off , ['qj', 'kd']), 'lj': (%lj:off , ['cx', 'zz']), 'th': (%th:off , ['bh']), 'rp': (%rp:off , ['bk', 'cm']), 'xk': (%xk:off , ['nv', 'kd']), 'qv': (%qv:off , ['ks']), 'mj': (%mj:off , ['xg', 'mh']), 'lh': (%lh:off , ['zz', 'pz']), 'kb': (%kb:off , ['hv', 'kd']), 'pg': (%pg:off , ['lz']), 'qm': (%qm:off , ['kb', 'kd']), 'pc': (%pc:off , ['cm', 'xl']), 'hv': (%hv:off , ['kd', 'rn']), 'fr': (%fr:off , ['fl', 'mh']), 'mp': (%mp:off , ['zz']), 'xl': (%xl:off , ['cm', 'gb']), 'tp': (%tp:off , ['mh']), 'gb': (%gb:off , ['rp']), 'pz': (%pz:off , ['mf', 'zz']), 'qn': (%qn:off , ['cm', 'sk']), 'fl': (%fl:off , ['tp', 'mh']), 'zq': (%zq:off , ['th',

KeyboardInterrupt: 